In [ ]:
!pip install dostoevsky

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.11.1-py3-none-any.whl (227 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4199773 sha256=13c5f6238b3f99f77d8a23436c2ce21e9fbf065840393584a224e305dda6ec0b
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [ ]:
!pip install nltk

In [ ]:
!pip install pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 32.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=a4d39207b005ecf311cc5ff0d5e7aa3fc61a20ea52b708343f1651c6fff0f8a8
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [ ]:
!pip install -U pymorphy2-dicts-ru

Функция для получение отзывов

In [ ]:
import requests
from dostoevsky.tokenization import RegexTokenizer
from dostoevsky.models import FastTextSocialNetworkModel
import nltk
from nltk import word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords
import string
import pymorphy2
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
TOKEN = 'CSZAYKW-2PH4GQV-PX5SC9X-2X2D4RF'

In [ ]:
# in: @id - movie id, @limit - max number of movie reviews
# out: reviews ({'docs': [review list], 'total': total_reviews,
# 'limit': @limit, 'page': @page_number, 'pages': total_pages})
def get_reviews_by_movie_id(id: int, limit: int, page_number: int = 1) -> dict:
    base_url = 'https://api.kinopoisk.dev/v1.4/review?'
    const_params = '&selectFields=id&selectFields=title&selectFields=type&selectFields=review&selectFields=date'
    params = f'page={page_number}&limit={limit}&movieId={id}'

    url = base_url + params + const_params
    response = requests.get(url, headers={'X-API-KEY': TOKEN}).json()

    rws = [x['review'] for x in response['docs']]
    ids = [x['id'] for x in response['docs']]
    types = [x['type'] for x in response['docs']]
    reviews = {'reviews': rws, 'ids': ids, 'true_types': types, 'pred_types': []}

    return reviews

In [ ]:
# remove stop words & punktuations, normalize words
def clear_data(response: list) -> list:
    # функция удаляет заданные символы из текста
    def replace_chars_in_text(text, chars, sub_char):
        return sub_char.join([ch for ch in text if ch not in chars])

    new_reviews = response['reviews']

    for i in range(len(new_reviews)):
        # привожу к нижнему регистру и удаляю пунктуацию
        new_reviews[i] = new_reviews[i].lower()
        new_reviews[i] = replace_chars_in_text(new_reviews[i], string.punctuation, '')

        # токенизирую
        text_tokens = word_tokenize(new_reviews[i])

        # нормализую слова
        morph = pymorphy2.MorphAnalyzer()

        for j in range(len(text_tokens)):
            text_tokens[j] = morph.parse(text_tokens[j])[0].normal_form

        # удаляю стоп-слова
        russian_stopwords = stopwords.words("russian")
        text_tokens = [word for word in text_tokens if not word in russian_stopwords]

        new_reviews[i] = ' '.join(text_tokens)

    response['reviews'] = new_reviews
    return response

Функция для оценки тональности отзывов

In [ ]:
def predict_review_type(reviews: dict) -> dict:
    tokenizer = RegexTokenizer()
    FastTextSocialNetworkModel.MODEL_PATH = '/home/fasttext-social-network-model.bin'
    model = FastTextSocialNetworkModel(tokenizer=tokenizer)
    results = model.predict(reviews['reviews'], k=2)

    for i in range(len(reviews['reviews'])):
        reviews['pred_types'].append(results[i])

    return reviews

In [ ]:
def show_review_type_predictions(reviews: dict):
    for i in range(len(reviews['reviews'])):
        review_id = reviews['ids'][i]
        true_type = reviews['true_types'][i]
        predicted_type = reviews['pred_types'][i]

        print(f'review id: {review_id}, true_type: {true_type}, predicted_type: {predicted_type}')

Главная функция

In [ ]:
# in: @id - movie id, @limit - max number of movie reviews
# out: reviews ({'docs': [review list], 'total': total_reviews,
# 'limit': @limit, 'page': @page_number, 'pages': total_pages})
def get_predicted_review_types(id: int, limit: int, page_number: int = 1) -> dict:
    reviews = get_reviews_by_movie_id(id, limit, page_number)
    reviews = clear_data(reviews)
    reviews = predict_review_type(reviews)

    return reviews

Оценка тональности отзывов

https://www.kinopoisk.ru/film/435/

In [ ]:
reviews1 = get_predicted_review_types(435, 20, 2)

In [ ]:
show_review_type_predictions(reviews1)

review id: 2882971, true_type: Негативный, predicted_type: {'neutral': 0.3629792034626007, 'negative': 0.275139719247818}
review id: 2876984, true_type: Позитивный, predicted_type: {'neutral': 0.4688006341457367, 'negative': 0.23935934901237488}
review id: 2871313, true_type: Позитивный, predicted_type: {'neutral': 0.4688006341457367, 'negative': 0.23935934901237488}
review id: 2855665, true_type: Позитивный, predicted_type: {'neutral': 0.5544804334640503, 'positive': 0.3140605390071869}
review id: 2847677, true_type: Позитивный, predicted_type: {'neutral': 0.5000100135803223, 'positive': 0.2568419873714447}
review id: 2820396, true_type: Позитивный, predicted_type: {'neutral': 0.4765896201133728, 'negative': 0.275139719247818}
review id: 2760013, true_type: Позитивный, predicted_type: {'neutral': 0.4301573634147644, 'positive': 0.3276783227920532}
review id: 2752464, true_type: Позитивный, predicted_type: {'neutral': 0.44553956389427185, 'positive': 0.23935934901237488}
review id: 273

https://www.kinopoisk.ru/film/471/

In [ ]:
reviews2 = get_predicted_review_types(471, 20, 2)

In [ ]:
show_review_type_predictions(reviews2)

review id: 2372267, true_type: Позитивный, predicted_type: {'neutral': 0.3702353835105896, 'negative': 0.2509227991104126}
review id: 2356594, true_type: Позитивный, predicted_type: {'positive': 0.320831298828125, 'skip': 0.21207880973815918}
review id: 2327847, true_type: Позитивный, predicted_type: {'neutral': 0.5621865391731262, 'positive': 0.1824355274438858}
review id: 2324351, true_type: Позитивный, predicted_type: {'positive': 0.3775506913661957, 'neutral': 0.23371636867523193}
review id: 2313113, true_type: Позитивный, predicted_type: {'neutral': 0.3702353835105896, 'positive': 0.23935934901237488}
review id: 2300657, true_type: Позитивный, predicted_type: {'positive': 0.4225146472454071, 'neutral': 0.3276783227920532}
review id: 2296459, true_type: Позитивный, predicted_type: {'positive': 0.4688006341457367, 'neutral': 0.15611489117145538}
review id: 2291610, true_type: Позитивный, predicted_type: {'neutral': 0.4921981394290924, 'positive': 0.22816647589206696}
review id: 2269

https://www.kinopoisk.ru/film/8161/

In [ ]:
reviews3 = get_predicted_review_types(8161, 20, 2)

In [ ]:
show_review_type_predictions(reviews3)

review id: 2094725, true_type: Позитивный, predicted_type: {'positive': 0.3415925204753876, 'neutral': 0.26285186409950256}
review id: 2076582, true_type: Позитивный, predicted_type: {'neutral': 0.38492217659950256, 'positive': 0.2509227991104126}
review id: 2047620, true_type: Негативный, predicted_type: {'neutral': 0.4225146472454071, 'negative': 0.2568419873714447}
review id: 2028985, true_type: Нейтральный, predicted_type: {'neutral': 0.29422497749328613, 'negative': 0.21207880973815918}
review id: 2006161, true_type: Позитивный, predicted_type: {'positive': 0.287777841091156, 'negative': 0.23935934901237488}
review id: 1911637, true_type: Позитивный, predicted_type: {'positive': 0.5312193632125854, 'neutral': 0.3140605390071869}
review id: 1909148, true_type: Позитивный, predicted_type: {'neutral': 0.59267657995224, 'negative': 0.1294127255678177}
review id: 1905434, true_type: Позитивный, predicted_type: {'neutral': 0.38492217659950256, 'positive': 0.287777841091156}
review id: 1

https://www.kinopoisk.ru/film/707/

In [ ]:
reviews4 = get_predicted_review_types(707, 20, 2)

In [ ]:
show_review_type_predictions(reviews4)

review id: 2161678, true_type: Позитивный, predicted_type: {'neutral': 0.4843900501728058, 'positive': 0.1968362182378769}
review id: 2084613, true_type: Позитивный, predicted_type: {'neutral': 0.3140605390071869, 'positive': 0.20182321965694427}
review id: 2066336, true_type: Позитивный, predicted_type: {'neutral': 0.585111141204834, 'positive': 0.1824355274438858}
review id: 2043551, true_type: Позитивный, predicted_type: {'neutral': 0.6992644667625427, 'positive': 0.15611489117145538}
review id: 2002061, true_type: Позитивный, predicted_type: {'positive': 0.275139719247818, 'negative': 0.1968362182378769}
review id: 1980112, true_type: Позитивный, predicted_type: {'neutral': 0.7122421860694885, 'positive': 0.14805719256401062}
review id: 1967130, true_type: Позитивный, predicted_type: {'positive': 0.41490885615348816, 'neutral': 0.3998216390609741}
review id: 1922748, true_type: Позитивный, predicted_type: {'positive': 0.30736804008483887, 'negative': 0.2814156115055084}
review id: 

https://www.kinopoisk.ru/film/327/

In [ ]:
reviews5 = get_predicted_review_types(327, 20, 2)

In [ ]:
show_review_type_predictions(reviews5)

review id: 2478152, true_type: Позитивный, predicted_type: {'positive': 0.3923468291759491, 'neutral': 0.2814156115055084}
review id: 2293405, true_type: Позитивный, predicted_type: {'neutral': 0.4843900501728058, 'negative': 0.21207880973815918}
review id: 2253774, true_type: Позитивный, predicted_type: {'neutral': 0.40734341740608215, 'positive': 0.21207880973815918}
review id: 2234359, true_type: Позитивный, predicted_type: {'neutral': 0.44553956389427185, 'negative': 0.20182321965694427}
review id: 2229918, true_type: Позитивный, predicted_type: {'neutral': 0.5000100135803223, 'negative': 0.19194278120994568}
review id: 2217812, true_type: Позитивный, predicted_type: {'neutral': 0.5698626637458801, 'negative': 0.1778208613395691}
review id: 2164336, true_type: Позитивный, predicted_type: {'positive': 0.3702353835105896, 'neutral': 0.3140605390071869}
review id: 2156135, true_type: Позитивный, predicted_type: {'positive': 0.5544804334640503, 'neutral': 0.1732981950044632}
review id:

In [ ]:
all_preds = {}

for key in reviews1.keys():
    all_preds[key] = reviews1[key] + reviews2[key] + reviews3[key] + reviews4[key] + reviews5[key]

In [ ]:
len(all_preds['ids'])

100

In [ ]:
for i in range(len(all_preds['pred_types'])):
    all_preds['pred_types'][i] = list(all_preds['pred_types'][i].keys())[0]

In [ ]:
all_preds['ids'][:5]

[2882971, 2876984, 2871313, 2855665, 2847677]

In [ ]:
all_preds['true_types'][:5]

['Негативный', 'Позитивный', 'Позитивный', 'Позитивный', 'Позитивный']

In [ ]:
all_preds['pred_types'][:5]

['neutral', 'neutral', 'neutral', 'neutral', 'neutral']

In [ ]:
all_preds_with_codes = all_preds

for i in range(len(all_preds['true_types'])):
    if all_preds['true_types'][i] == 'Негативный':
        all_preds['true_types'][i] = 0
    elif all_preds['true_types'][i] == 'Позитивный':
        all_preds['true_types'][i] = 1
    else:
        all_preds['true_types'][i] = 2

    if all_preds['pred_types'][i] == 'positive':
        all_preds['pred_types'][i] = 0
    elif all_preds['pred_types'][i] == 'negative':
        all_preds['pred_types'][i] = 1
    else:
        all_preds['pred_types'][i] = 2

In [ ]:
all_preds['true_types'][:5]

[0, 1, 1, 1, 1]

In [ ]:
all_preds['pred_types'][:5]

[2, 2, 2, 2, 2]